# **--- CNN MODEL ---** #

## **I. Libraries import** ##


In [122]:
import os
from pathlib import Path
import shutil
import zipfile
import requests
from tqdm import tqdm
import random
import mlflow

# Torch ------------------
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models,datasets
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

# Metrics ------------------
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

# Visualization ---------
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image


from dotenv import load_dotenv 

load_dotenv()


True

We select the appropriate torch device

In [123]:
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # Apple M1/M2/M3
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device")

Using mps device


## **II. Images import and processing** ##


### Load dataset ###


In [124]:
DATASET_NAME=os.getenv("DATASET_NAME","inrae")
DATASET_DIR=Path(f"../data-{DATASET_NAME}")

In [125]:
#  We randomly reduce the amount of images in the "healthy" class that is too represented : 

random.seed(42)

TARGET_NB = 350
sain = Path(f"{DATASET_DIR}/raw_data_inrae/sain")

images = list(sain.glob("*"))
n_images = len(images)

print(f"{sain.name}: {n_images} images")

if n_images <= TARGET_NB:
    print("Nothing to delete")
else:
    images_to_delete = random.sample(
        images, n_images - TARGET_NB
    )

    for img in images_to_delete:
        img.unlink()

    print(f"{len(images_to_delete)} images deleted")

sain: 350 images
Nothing to delete


In [126]:
#  Clean the target folder
organized_data = Path(f"{DATASET_DIR}/organized_data_inrae")

if organized_data.exists():
    shutil.rmtree(organized_data)


organized_data.mkdir(parents=True, exist_ok=True)

# --------------   Divide the dataset into Train, Val and Test -------------------
random.seed(42)

raw_data_path = Path(f"{DATASET_DIR}/raw_data_inrae")
organized_data = Path(f"{DATASET_DIR}/organized_data_inrae")

SPLITS = {
    "train": 0.7,
    "val": 0.15,
    "test": 0.15
}

# Create folder structure
for split in SPLITS:
    for class_dir in raw_data_path.iterdir():
        if class_dir.is_dir():
            (organized_data / split / class_dir.name).mkdir(parents=True, exist_ok=True)

# Split images
for class_dir in raw_data_path.iterdir():
    if not class_dir.is_dir():
        continue

    images = [
        img for img in class_dir.iterdir()
        if img.suffix.lower() in {".jpg", ".jpeg", ".png"}
    ]

    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * SPLITS["train"])
    n_val = int(n_total * SPLITS["val"])

    train_imgs = images[:n_train]
    val_imgs = images[n_train:n_train + n_val]
    test_imgs = images[n_train + n_val:]

    for img in train_imgs:
        shutil.copy(img, organized_data / "train" / class_dir.name / img.name)

    for img in val_imgs:
        shutil.copy(img, organized_data / "val" / class_dir.name / img.name)

    for img in test_imgs:
        shutil.copy(img, organized_data / "test" / class_dir.name / img.name)



In [127]:
# We check the size of the new folders

ROOT_DIR = Path(f"{DATASET_DIR}/organized_data_inrae")

for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    total = 0

    for class_dir in (ROOT_DIR / split).iterdir():
        if class_dir.is_dir():
            n_files = len(list(class_dir.glob("*")))
            total += n_files
            print(f"  {class_dir.name:<20} : {n_files}")

    print(f"  TOTAL {split:<14} : {total}")




TRAIN
  guignardia_bidwellii : 129
  elsinoe_ampelina     : 202
  erysiphe_necator     : 71
  sain                 : 244
  plasmopara_viticola  : 260
  phaeomoniella_chlamydospora : 65
  colomerus_vitis      : 90
  TOTAL train          : 1061

VAL
  guignardia_bidwellii : 27
  elsinoe_ampelina     : 43
  erysiphe_necator     : 15
  sain                 : 52
  plasmopara_viticola  : 55
  phaeomoniella_chlamydospora : 14
  colomerus_vitis      : 19
  TOTAL val            : 225

TEST
  guignardia_bidwellii : 29
  elsinoe_ampelina     : 44
  erysiphe_necator     : 16
  sain                 : 54
  plasmopara_viticola  : 57
  phaeomoniella_chlamydospora : 15
  colomerus_vitis      : 20
  TOTAL test           : 235


### Pipeline for data transformation ###


In [128]:
# Transformations of the train set with data augmentation

transform_train= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(-45,+45)),
    transforms.ToTensor(),        
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # Normalization with the values for the pre-trained Resnet model
])

In [129]:
# Create the path to the training dataset
data_train = Path(f"{DATASET_DIR}/organized_data_inrae/train")

# Load dataset with ImageFolder
train_dataset = ImageFolder(root=data_train, transform=transform_train)

# Get class names
class_names = train_dataset.classes

In [130]:
#  Transformation pipeline without data augmentation for the validation and the test set

transform= transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionnement à 224x224
    transforms.ToTensor(),          # Conversion en tenseur
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # Normalisation avec les valeurs du modèle pré-entraîné Resnet
])

# Create the path to the training dataset
data_val = Path(f"{DATASET_DIR}/organized_data_inrae/val")
data_test = Path(f"{DATASET_DIR}/organized_data_inrae/test")

# Load dataset with ImageFolder
val_dataset = ImageFolder(root=data_val,transform=transform)
test_dataset = ImageFolder(root=data_test, transform=transform)


In [131]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## **Fine tuning (Resnet18)** ##

### Preparing the MLFlow tracking ###

In [109]:
# Set tracking URI to your Hugging Face application
MLFLOW_URI=os.getenv('MLFLOW_URI',"https://gviel-mlflow37.hf.space/")
mlflow.set_tracking_uri(os.environ["MLFLOW_URI"])

# Set experiment's info
EXPERIMENT_NAME= os.getenv('EXPERIMENT_NAME',"Vitiscan_CNN_MLFlow_FineTuning")+"FINE_TUNING"
mlflow.set_experiment(EXPERIMENT_NAME)

# Get our experiment info
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

### Importing a pre-trained model

In [110]:
#model = models.resnet18(pretrained=True) # pour version <0.13.0
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT) # pour version >=0.13.0
model = model.to(device)
print(f"Model device: {next(model.parameters()).device}")
print(device)

Model device: mps:0
mps


### Replace classification layer to adapt the model to our features ###

In [111]:
nb_classes=len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, nb_classes)


### Freeze the feature extraction layers ###

In [112]:
# We freeze the entire network
for param in model.parameters():
    param.requires_grad = False

# We unfreeze only the last Resnet18 blocks of layers (called "Layer4 for resnet18")
# AND we unfreeze the classifier fc
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

In [113]:
# Print model summary
summary(model, input_size=(1, 3, 224, 224))  # (batch_size, input_features)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 7]                    --
├─Conv2d: 1-1                            [1, 64, 112, 112]         (9,408)
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         (128)
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]   

### Defining the cost function and optimizer ###

In [114]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device) 

learning_rate=0.0001

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
    weight_decay=0.0001 # Ridge regulation to avoid overfitting
)

### Train the model ###

We check an output on the fist batch

In [115]:
# Model outputs
image, label = next(iter(train_loader))
image=image.to(device)
model=model.to(device)
logit= model(image) # Resnet18 output
logit
# Attention il faut que l'image et le modele aient le même device

tensor([[ 5.6696e-01,  1.1915e+00,  3.9915e-01, -5.1671e-01, -4.6692e-01,
          1.4614e-01, -1.2787e+00],
        [ 9.6687e-01,  1.0571e+00, -3.8468e-01, -6.1200e-01, -1.9921e-01,
          3.9960e-01, -9.8709e-01],
        [-6.0490e-02,  1.1982e+00,  5.6133e-01, -7.5117e-01, -5.2219e-01,
         -3.8251e-01, -8.5225e-01],
        [-2.0685e-01,  5.7663e-01,  3.3269e-01, -4.3178e-01, -5.3747e-01,
          1.1735e-01, -1.8142e+00],
        [ 2.3610e-01,  1.1594e+00, -4.9645e-01, -8.4795e-01, -6.9856e-01,
         -1.3576e-01, -1.9910e+00],
        [ 2.5394e-01,  4.2684e-01, -2.1686e-01, -8.3230e-01, -3.7474e-01,
         -4.3500e-02, -1.3902e+00],
        [-3.3378e-01,  1.0401e+00,  9.7580e-02, -3.1863e-01, -4.1905e-01,
         -5.3801e-01, -7.7645e-01],
        [ 5.2134e-01,  1.4445e+00,  7.2755e-02, -3.8107e-01, -4.5879e-01,
         -5.6436e-01, -1.9546e+00],
        [ 1.5829e-01,  6.7098e-01,  6.6299e-02, -7.2604e-01,  2.2335e-01,
         -7.0277e-01, -1.2803e+00],
        [ 

In [116]:
class_names

['colomerus_vitis',
 'elsinoe_ampelina',
 'erysiphe_necator',
 'guignardia_bidwellii',
 'phaeomoniella_chlamydospora',
 'plasmopara_viticola',
 'sain']

In [117]:
def evaluate_model_on_test(model, test_loader, device):
    
    model.eval() # Mode évaluation
    y_true = []
    y_pred = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            logit = model(images)
            preds = logit.argmax(dim=1)
            
            # Compter l'exactitude
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            
            # Collecter pour les métriques de Scikit-learn
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    test_accuracy = correct / total
    return test_accuracy, y_true, y_pred

In [120]:
# Training function for a PyTorch model
def train(model, train_loader, val_loader, test_loader, criterion, optimizer, experiment, epochs=20, patience=5):
    
    # Early stopping variables
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    # We start a MLflow run
    with mlflow.start_run(experiment_id=experiment.experiment_id):

        # Logging Pytorch parameters into MLflow 
        params = {
            "optimizer": type(optimizer).__name__,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "epochs": epochs,
            "criterion": type(criterion).__name__,
            "model_architecture": type(model).__name__,
            "training_device": str(device),
            "weight_decay": optimizer.param_groups[0]["weight_decay"]
        }
        
        mlflow.log_params(params=params)
        mlflow.pytorch.autolog()

        # Dictionary to store loss and accuracy values for each epoch
        history = {
            'loss': [],
            'val_loss': [],
            'accuracy': [],
            'val_accuracy': []
        }

        # ------------------- TRAINING LOOP -------------------
        for epoch in range(epochs):
            model.train()
            total_loss, correct = 0, 0  

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                logit = model(images)
                loss = criterion(logit, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                correct += (logit.argmax(dim=1) == labels).sum().item()

            train_loss = total_loss / len(train_loader)
            train_acc = correct / len(train_loader.dataset)

            # ------------------- VALIDATION LOOP -------------------
            model.eval()
            val_loss, val_correct = 0, 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    logit = model(images)
                    loss = criterion(logit, labels)
                    val_loss += loss.item()
                    val_correct += (logit.argmax(dim=1) == labels).sum().item()

            val_loss /= len(val_loader)
            val_acc = val_correct / len(val_loader.dataset)

            # --- Save metrics ---
            history['loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['accuracy'].append(train_acc)
            history['val_accuracy'].append(val_acc)

            print(
                f"Epoch [{epoch+1}/{epochs}], "
                f"Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
            )

            #  Logging metrics
            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("train_accuracy", train_acc, step=epoch)
            mlflow.log_metric("validation_loss", val_loss, step=epoch)
            mlflow.log_metric("validation_accuracy", val_acc, step=epoch)

            # --- Early stopping check ---
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                best_model_state = model.state_dict()  # save the best model
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    model.load_state_dict(best_model_state)  # restore best model
                    break
                

        # ================= FINAL VALIDATION EVALUATION =================
        
        print("\n--- Final evaluation on the VALIDATION set ---\n")

        y_true_val, y_pred_val = [], []
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                logit = model(images)
                preds = logit.argmax(dim=1)

                y_true_val.extend(labels.cpu().numpy())
                y_pred_val.extend(preds.cpu().numpy())

        # --- Precision & Recall (Validation) ---
        precision_weighted_val = precision_score(
            y_true_val, y_pred_val, average='weighted', zero_division=0
        )
        precision_macro_val = precision_score(
            y_true_val, y_pred_val, average='macro', zero_division=0
        )

        recall_weighted_val = recall_score(
            y_true_val, y_pred_val, average='weighted', zero_division=0
        )
        recall_macro_val = recall_score(
            y_true_val, y_pred_val, average='macro', zero_division=0
        )

        mlflow.log_metric("Validation_precision_weighted", precision_weighted_val)
        mlflow.log_metric("Validation_precision_macro", precision_macro_val)
        mlflow.log_metric("Validation_recall_weighted", recall_weighted_val)
        mlflow.log_metric("Validation_recall_macro", recall_macro_val)

        # --- F1 Score (Validation) ---
        f1_weighted_val = f1_score(y_true_val, y_pred_val, average='weighted')
        f1_macro_val = f1_score(y_true_val, y_pred_val, average='macro')

        mlflow.log_metric("Validation_f1_score_weighted", f1_weighted_val)
        mlflow.log_metric("Validation_f1_score_macro", f1_macro_val)
        mlflow.log_metric("final_validation_accuracy", val_acc)

        # Printing metrics
        print(f"Precision Validation (weighted): {precision_weighted_val:.4f}")
        print(f"Recall Validation (weighted): {recall_weighted_val:.4f}")
        print(f"F1 Score Validation (weighted): {f1_weighted_val:.4f}")

        # --- Confusion Matrix (Validation) ---
        cm = confusion_matrix(y_true_val, y_pred_val)

        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm, annot=True, fmt='d', cmap='Blues'
        )

        plt.xticks(ticks= range(nb_classes),labels=class_names, rotation=45, ha="right")
        plt.yticks(ticks= range(nb_classes),labels=class_names,rotation=0)
        plt.ylabel('True class')
        plt.xlabel('Predicted class')
        plt.title('Confusion matrix - VALIDATION')
        plt.tight_layout()
        plt.savefig('confusion_matrix_VALIDATION.png', dpi=150)

        mlflow.log_artifact('confusion_matrix_VALIDATION.png')
        plt.close()


        # ================= FINAL TEST EVALUATION =================
        
        print("\n--- Final evaluation on the TEST set ---\n")

        test_acc, y_true_test, y_pred_test = evaluate_model_on_test(
            model, test_loader, device
        )

        # --- Precision & Recall (Test) ---
        precision_weighted_test = precision_score(
            y_true_test, y_pred_test, average='weighted', zero_division=0
        )
        precision_macro_test = precision_score(
            y_true_test, y_pred_test, average='macro', zero_division=0
        )

        recall_weighted_test = recall_score(
            y_true_test, y_pred_test, average='weighted', zero_division=0
        )
        recall_macro_test = recall_score(
            y_true_test, y_pred_test, average='macro', zero_division=0
        )

        mlflow.log_metric("Test_precision_weighted", precision_weighted_test)
        mlflow.log_metric("Test_precision_macro", precision_macro_test)
        mlflow.log_metric("Test_recall_weighted", recall_weighted_test)
        mlflow.log_metric("Test_recall_macro", recall_macro_test)

        # --- F1 Score (Test) ---
        f1_weighted_test = f1_score(y_true_test, y_pred_test, average='weighted')
        f1_macro_test = f1_score(y_true_test, y_pred_test, average='macro')

        mlflow.log_metric("Test_accuracy", test_acc)
        mlflow.log_metric("Test_f1score_weighted", f1_weighted_test)
        mlflow.log_metric("Test_f1score_macro", f1_macro_test)

        print(f"Test Accuracy: {test_acc:.4f}")
        print(f"Precision Test (weighted): {precision_weighted_test:.4f}")
        print(f"Recall Test (weighted): {recall_weighted_test:.4f}")
        print(f"F1 Score Test (weighted): {f1_weighted_test:.4f}")

        # --- Confusion Matrix (Test) ---
        cm_test = confusion_matrix(y_true_test, y_pred_test)

        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm_test, annot=True, fmt='d', cmap='Blues',
        )
        plt.xticks(ticks= range(nb_classes),labels=class_names, rotation=45, ha="right")
        plt.yticks(ticks= range(nb_classes),labels=class_names,rotation=0)

        plt.ylabel('True class')
        plt.xlabel('Predicted class')
        plt.title('Confusion matrix - TEST')
        plt.tight_layout()
        plt.savefig('confusion_matrix_TEST.png', dpi=150)

        mlflow.log_artifact('confusion_matrix_TEST.png')
        plt.close()

        # --- Model logging ---
        mlflow.pytorch.log_model(
            pytorch_model=model,
            artifact_path="Resnet18",
            registered_model_name=f"{params['model_architecture']}"
        )

        print("\n--- Metrics and model logged into MLflow ---\n")

        return history


In [96]:
DISEASES = {
    "colomerus_vitis" : "erinose",
    "elsinoe_ampelina" : "anthracnose",
    "erysiphe_necator":"oidium",
    "guignardia_bidwellii" : "pourriture_noire",
    "phaeomoniella_chlamydospora" : "esca",
    "plasmopara_viticola":"mildiou",
    "sain" : "sain"
    }

In [121]:
# Train the model and store the training history
history = train(model, train_loader, val_loader, test_loader, criterion, optimizer, experiment, epochs=40,patience=5)


Epoch [1/40], Loss: 0.0786, Acc: 0.9764, Val Loss: 0.2521, Val Acc: 0.9333
Epoch [2/40], Loss: 0.0363, Acc: 0.9915, Val Loss: 0.2337, Val Acc: 0.9422
Epoch [3/40], Loss: 0.0531, Acc: 0.9915, Val Loss: 0.2415, Val Acc: 0.9333
Epoch [4/40], Loss: 0.0440, Acc: 0.9868, Val Loss: 0.2470, Val Acc: 0.9156
Epoch [5/40], Loss: 0.0482, Acc: 0.9906, Val Loss: 0.2234, Val Acc: 0.9422
Epoch [6/40], Loss: 0.0516, Acc: 0.9906, Val Loss: 0.2114, Val Acc: 0.9333
Epoch [7/40], Loss: 0.0404, Acc: 0.9915, Val Loss: 0.2430, Val Acc: 0.9378
Epoch [8/40], Loss: 0.0228, Acc: 0.9934, Val Loss: 0.2340, Val Acc: 0.9467
Epoch [9/40], Loss: 0.0193, Acc: 0.9962, Val Loss: 0.2222, Val Acc: 0.9378
Epoch [10/40], Loss: 0.0191, Acc: 0.9953, Val Loss: 0.2583, Val Acc: 0.9467
Epoch [11/40], Loss: 0.0724, Acc: 0.9953, Val Loss: 0.2281, Val Acc: 0.9333
Early stopping triggered after 11 epochs

--- Final evaluation on the VALIDATION set ---

Precision Validation (weighted): 0.9331
Recall Validation (weighted): 0.9333
F1 Sco

2025/12/19 15:12:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'ResNet' already exists. Creating a new version of this model...
2025/12/19 15:13:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ResNet, version 22
Created version '22' of model 'ResNet'.



--- Metrics and model logged into MLflow ---

🏃 View run clean-goose-48 at: https://gviel-mlflow37.hf.space/#/experiments/4/runs/366fd4b021f440aeb5db947fa7717d4e
🧪 View experiment at: https://gviel-mlflow37.hf.space/#/experiments/4


### Visualization of the learning process ###

In [ ]:
from plotly import graph_objects as go
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

fig = go.Figure(data=[
                      go.Scatter(
                          y=history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'
)
fig.show()

: 

In [ ]:
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

fig = go.Figure(data=[
                      go.Scatter(
                          y=history["accuracy"],
                          name="Training Accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history["val_accuracy"],
                          name="Validation Accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val Accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'
)
fig.show()

: 